# Exotic Options Project - Workspace Setup

This notebook demonstrates the setup and basic usage of the exotic options project with scientific computing libraries.

## Overview
- Set up Python environment with Anaconda
- Demonstrate core scientific libraries
- Basic options pricing examples
- Data visualization with matplotlib and plotly

## 1. Install Required Packages

First, ensure all required packages are installed. If you haven't already, create the conda environment:

```bash
conda env create -f environment.yml
conda activate exotic-options
```

Or install packages using pip:

```bash
pip install -r requirements.txt
```

In [1]:
# Import essential scientific computing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from scipy import stats
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Configure matplotlib
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ All packages imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Matplotlib version: {matplotlib.__version__}")

✅ All packages imported successfully!
NumPy version: 2.2.6
Pandas version: 2.3.2


NameError: name 'matplotlib' is not defined

## 2. Configure Development Environment

Set up the development environment with proper configurations and path settings.

In [ ]:
import sys
import os
from pathlib import Path

# Add project root to Python path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Create data directory if it doesn't exist
data_dir = project_root / 'data'
data_dir.mkdir(exist_ok=True)

print(f"Project root: {project_root}")
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")
print(f"Working directory: {os.getcwd()}")

## 3. Set Up Version Control

Initialize Git repository and configure settings (if not already done).

In [ ]:
# Check if we're in a Git repository
import subprocess

def run_git_command(command):
    try:
        result = subprocess.run(command, shell=True, capture_output=True, text=True, cwd=project_root)
        return result.returncode == 0, result.stdout.strip(), result.stderr.strip()
    except Exception as e:
        return False, "", str(e)

# Check Git status
success, output, error = run_git_command("git status")
if success:
    print("✅ Git repository is already initialized")
    print(f"Git status: {output.split(chr(10))[0]}")
else:
    print("ℹ️  Git repository not initialized")
    print("To initialize Git, run: git init")

# Show current Git config
success, output, error = run_git_command("git config --list --local")
if success and output:
    print("\n📋 Local Git configuration:")
    for line in output.split('\n')[:5]:  # Show first 5 lines
        print(f"   {line}")

## 4. Create Project Structure

Verify and display the project structure.

In [ ]:
def display_project_structure(path, prefix="", max_depth=3, current_depth=0):
    """Display project directory structure."""
    if current_depth > max_depth:
        return
    
    items = []
    try:
        items = sorted(path.iterdir(), key=lambda x: (x.is_file(), x.name.lower()))
    except PermissionError:
        return
    
    for i, item in enumerate(items):
        if item.name.startswith('.'):
            continue
            
        is_last = i == len([x for x in items if not x.name.startswith('.')]) - 1
        current_prefix = "└── " if is_last else "├── "
        print(f"{prefix}{current_prefix}{item.name}")
        
        if item.is_dir() and current_depth < max_depth:
            next_prefix = prefix + ("    " if is_last else "│   ")
            display_project_structure(item, next_prefix, max_depth, current_depth + 1)

print("📁 Project Structure:")
print(f"{project_root.name}/")
display_project_structure(project_root)

## 5. Configure IDE Settings

Display current Jupyter and Python environment settings.

In [ ]:
import jupyter_core
import IPython

print("🔧 IDE and Environment Configuration:")
print(f"Jupyter core version: {jupyter_core.__version__}")
print(f"IPython version: {IPython.__version__}")

# Display Python packages relevant to scientific computing
important_packages = ['numpy', 'pandas', 'matplotlib', 'scipy', 'plotly', 'seaborn']
print("\n📦 Key Scientific Computing Packages:")

for package in important_packages:
    try:
        module = __import__(package)
        version = getattr(module, '__version__', 'unknown')
        print(f"   ✅ {package}: {version}")
    except ImportError:
        print(f"   ❌ {package}: not installed")

# Display current conda environment (if using conda)
conda_env = os.environ.get('CONDA_DEFAULT_ENV', 'Not using conda')
print(f"\n🐍 Active environment: {conda_env}")

## 6. Set Up Virtual Environment

Verify the virtual environment setup and display environment information.

In [ ]:
# Check virtual environment
print("🌍 Virtual Environment Information:")
print(f"Python executable: {sys.executable}")
print(f"Virtual environment: {os.environ.get('VIRTUAL_ENV', 'None')}")
print(f"Conda environment: {os.environ.get('CONDA_DEFAULT_ENV', 'None')}")
print(f"Conda prefix: {os.environ.get('CONDA_PREFIX', 'None')}")

# Display installed packages count
try:
    import pkg_resources
    installed_packages = list(pkg_resources.working_set)
    print(f"\n📊 Total installed packages: {len(installed_packages)}")
    
    # Show first few packages as sample
    print("\nSample of installed packages:")
    for pkg in sorted(installed_packages, key=lambda x: x.key)[:10]:
        print(f"   {pkg.key}: {pkg.version}")
        
except ImportError:
    print("pkg_resources not available")

print("\n✅ Environment setup complete!")

## Demo: Basic Options Pricing

Now let's demonstrate the project capabilities with some basic options pricing examples.

In [ ]:
# Import our custom options module
try:
    from src.options import BlackScholesModel, monte_carlo_simulation
    print("✅ Successfully imported custom options module")
except ImportError as e:
    print(f"⚠️  Could not import custom module: {e}")
    print("Defining basic functions inline...")
    
    class BlackScholesModel:
        def __init__(self, S0, r, sigma, T):
            self.S0, self.r, self.sigma, self.T = S0, r, sigma, T
        
        def european_call_price(self, K):
            d1 = (np.log(self.S0/K) + (self.r + 0.5*self.sigma**2)*self.T) / (self.sigma*np.sqrt(self.T))
            d2 = d1 - self.sigma*np.sqrt(self.T)
            return self.S0*stats.norm.cdf(d1) - K*np.exp(-self.r*self.T)*stats.norm.cdf(d2)

# Set up option parameters
S0 = 100    # Current stock price
K = 100     # Strike price
r = 0.05    # Risk-free rate
sigma = 0.2 # Volatility
T = 1.0     # Time to maturity (1 year)

# Create Black-Scholes model
bs_model = BlackScholesModel(S0, r, sigma, T)
call_price = bs_model.european_call_price(K)

print(f"📊 Black-Scholes European Call Option Price:")
print(f"   Stock Price (S0): ${S0}")
print(f"   Strike Price (K): ${K}")
print(f"   Risk-free Rate: {r*100}%")
print(f"   Volatility: {sigma*100}%")
print(f"   Time to Maturity: {T} year")
print(f"   Call Price: ${call_price:.2f}")

## Demo: Monte Carlo Simulation

Demonstrate Monte Carlo simulation for stock price paths.

In [ ]:
# Monte Carlo simulation
n_paths = 5000
n_steps = 252  # Daily steps for 1 year

# Simple Monte Carlo function if module import failed
def simple_monte_carlo(S0, r, sigma, T, n_paths=1000, n_steps=252):
    dt = T / n_steps
    paths = np.zeros((n_paths, n_steps + 1))
    paths[:, 0] = S0
    
    for i in range(1, n_steps + 1):
        z = np.random.standard_normal(n_paths)
        paths[:, i] = paths[:, i-1] * np.exp((r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*z)
    
    return paths

# Generate stock price paths
try:
    stock_paths = monte_carlo_simulation(S0, r, sigma, T, n_paths, n_steps)
except NameError:
    stock_paths = simple_monte_carlo(S0, r, sigma, T, n_paths, n_steps)

# Calculate final stock prices
final_prices = stock_paths[:, -1]

# Calculate call option payoffs
call_payoffs = np.maximum(final_prices - K, 0)
mc_call_price = np.mean(call_payoffs) * np.exp(-r * T)

print(f"🎲 Monte Carlo Simulation Results ({n_paths:,} paths):")
print(f"   Analytical Call Price: ${call_price:.2f}")
print(f"   Monte Carlo Call Price: ${mc_call_price:.2f}")
print(f"   Difference: ${abs(call_price - mc_call_price):.2f}")
print(f"   Final Stock Price - Mean: ${np.mean(final_prices):.2f}")
print(f"   Final Stock Price - Std: ${np.std(final_prices):.2f}")

## Visualization: Stock Price Paths

Create visualizations using matplotlib and plotly.

In [ ]:
# Create time axis
time_axis = np.linspace(0, T, n_steps + 1)

# Plot with matplotlib
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Sample stock price paths
n_sample_paths = 50
for i in range(n_sample_paths):
    ax1.plot(time_axis, stock_paths[i], alpha=0.3, linewidth=0.8)

# Add mean path
mean_path = np.mean(stock_paths, axis=0)
ax1.plot(time_axis, mean_path, 'r-', linewidth=2, label='Mean Path')
ax1.axhline(y=S0, color='k', linestyle='--', alpha=0.5, label='Initial Price')
ax1.set_title(f'Monte Carlo Stock Price Paths (Sample of {n_sample_paths})')
ax1.set_xlabel('Time (Years)')
ax1.set_ylabel('Stock Price ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Final price distribution
ax2.hist(final_prices, bins=50, alpha=0.7, density=True, color='skyblue', edgecolor='black')
ax2.axvline(x=np.mean(final_prices), color='r', linestyle='-', linewidth=2, label=f'Mean: ${np.mean(final_prices):.1f}')
ax2.axvline(x=K, color='orange', linestyle='--', linewidth=2, label=f'Strike: ${K}')
ax2.set_title('Distribution of Final Stock Prices')
ax2.set_xlabel('Final Stock Price ($)')
ax2.set_ylabel('Density')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"📈 Created matplotlib visualization with {n_sample_paths} sample paths")

## Interactive Visualization with Plotly

Create an interactive plot using Plotly.

In [ ]:
# Create interactive plotly visualization
fig = go.Figure()

# Add sample paths
n_interactive_paths = 20
for i in range(n_interactive_paths):
    fig.add_trace(go.Scatter(
        x=time_axis,
        y=stock_paths[i],
        mode='lines',
        line=dict(width=1, color='lightblue'),
        showlegend=False,
        hovertemplate='Path %{trace}: $%{y:.2f}<extra></extra>'
    ))

# Add mean path
fig.add_trace(go.Scatter(
    x=time_axis,
    y=mean_path,
    mode='lines',
    line=dict(width=3, color='red'),
    name='Mean Path',
    hovertemplate='Mean Path: $%{y:.2f}<extra></extra>'
))

# Add initial price line
fig.add_hline(y=S0, line_dash="dash", line_color="black", 
              annotation_text=f"Initial Price: ${S0}")

# Add strike price line
fig.add_hline(y=K, line_dash="dot", line_color="orange", 
              annotation_text=f"Strike Price: ${K}")

fig.update_layout(
    title=f'Interactive Monte Carlo Stock Price Simulation<br><sub>({n_interactive_paths} paths shown out of {n_paths:,} total)</sub>',
    xaxis_title='Time (Years)',
    yaxis_title='Stock Price ($)',
    hovermode='x unified',
    template='plotly_white',
    height=500
)

fig.show()

print(f"📊 Created interactive Plotly visualization")
print(f"   Hover over the lines to see values")
print(f"   Use the toolbar to zoom, pan, and explore")

## Summary

This notebook has successfully demonstrated:

✅ **Environment Setup**: Configured Python environment with scientific computing libraries  
✅ **Project Structure**: Verified project organization and file structure  
✅ **Options Pricing**: Implemented Black-Scholes model for European options  
✅ **Monte Carlo**: Performed Monte Carlo simulation for stock price paths  
✅ **Visualization**: Created static and interactive plots  

## Next Steps

1. **Explore Exotic Options**: Implement pricing models for barrier options, Asian options, etc.
2. **Advanced Analytics**: Add Greeks calculation, risk metrics, and sensitivity analysis
3. **Data Integration**: Connect to market data sources for real-time pricing
4. **Portfolio Optimization**: Implement portfolio optimization techniques
5. **Machine Learning**: Apply ML techniques for options pricing and risk modeling

The workspace is now ready for advanced quantitative finance development! 🚀